# Import libraries

In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics
from scipy import stats
from scipy.stats import skew 
from sklearn.model_selection import train_test_split
# calculate error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import math

import time

# Normal eqn functions

In [2]:
def w_normaleqn(X,y):
    a = X.T.dot(X)
    b = X.T.dot(y)
    w = np.linalg.inv(a).dot(b)
    return w
    
# numpy. linalg. inv()

def mseloss_normaleqn(X,w,y):
    n = len(X)
    Xwy = X.dot(w)-y
    loss= 1/n*Xwy.T.dot(Xwy)
    return loss

In [3]:
# not sure if this is needed tbh
def f_pred(X,w):
    f = X.dot(w)
    return f 

def grad_loss_normaleqn(y_pred,y,X):
    n = len(y)
    grad = 2/n*(X.T.dot(y_pred)-X.T.dot(y))
    return grad

## (copied from the pre-processing file)
I lazy<br>
Try normal eqn with the different combis of pre-processing

FROM EDA:

LotFrontage /number of null data: 259

MasVnrArea /number of null data: 8

Electrical /number of null data: 1

GarageYrBlt /number of null data: 81

For feature with too many missing values, remove this feature entirely.

Set treshold to be 10% of all data points
For feature with only a few missing values, remove that data point.

In [4]:
train0 = pd.read_csv('train.csv')

# Remove null
nullData = [['LotFrontage', 259], ['MasVnrArea', 8], ['Electrical', 1], ['GarageYrBlt', 81]]
n = len(train0)
treshold = 0.1
drop = []

print('Drop feature - too many nulls:')
for i in nullData:
    if i[1]/n > treshold: # Arbitrary treshold: 10%
        print(i[0])
        train0.drop(columns=[i[0]], inplace=True)
    else:
        drop.append(i[0])
        
print('Remove data point:')
print(drop)
train0.dropna(subset=drop, inplace=True)

# OHE
categorical = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond','Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
train0 = pd.get_dummies(train0, columns= categorical, prefix= categorical)

train0.head()

Drop feature - too many nulls:
LotFrontage
Remove data point:
['MasVnrArea', 'Electrical', 'GarageYrBlt']


,Id,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,8450,2003,2003,196.0,706,0,150,856,856,...,0,0,0,1,0,0,0,0,1,0
1,2,9600,1976,1976,0.0,978,0,284,1262,1262,...,0,0,0,1,0,0,0,0,1,0
2,3,11250,2001,2002,162.0,486,0,434,920,920,...,0,0,0,1,0,0,0,0,1,0
3,4,9550,1915,1970,0.0,216,0,540,756,961,...,0,0,0,1,1,0,0,0,0,0
4,5,14260,2000,2000,350.0,655,0,490,1145,1145,...,0,0,0,1,0,0,0,0,1,0


# Four Standard Basic Optimisations
In order to determine which learning algorithm is the most accurate for this application, we will run each one on the dataset with a few basic optimisations.
For the purpose of preliminary comparisons, we will process the data in 4 general ways and save them to individual csv files. The 4 general ways are as follows (in order):
<br><br>
<u>ONE</u>
- Dealing with null data
- Dealing with highly-skewed categorical data
- One Hot Encoding
- Feature selection

<u>TWO</u>
- Dealing with null data
- Dealing with highly-skewed categorical data
- One Hot Encoding
- <b>Dealing with highly-skewed continuous data</b>
- Feature selection

<u>THREE</u>
- Dealing with null data
- Dealing with highly-skewed categorical data
- One Hot Encoding
- Dealing with highly-skewed continuous data
- <b>Normalisation</b>
- Feature selection

<u>FOUR</u>
- Dealing with null data
- Dealing with highly-skewed categorical data
- One Hot Encoding
- Dealing with highly-skewed continuous data
- <b>Z-scoring</b>
- Feature selection

<br><br>
(Note: Feature selection is done at the end of each version. The rest will build upon ONE or TWO)

In [5]:
#importing the csv datasets
print('train0 shape:', train0.shape)

train1 = pd.read_csv('train_ONE.csv')
print('train1 shape:', train1.shape)

train2 = pd.read_csv('train_TWO.csv')
print('train2 shape:', train2.shape)

train3 = pd.read_csv('train_THREE.csv')
print('train3 shape:', train3.shape)

train4 = pd.read_csv('train_FOUR.csv')
print('train4 shape:', train4.shape)

train0 shape: (1370, 316)
train1 shape: (1370, 70)
train2 shape: (688, 75)
train3 shape: (688, 75)
train4 shape: (688, 75)


## Linear regression

### Train 0

In [6]:
print('LINEAR REGRESSION - train0:')
X0 = train0.copy()
X0.drop(['Id', 'SalePrice'], axis=1, inplace=True)
y0 = train0['SalePrice'][:,None]

train_X0, test_X0, train_y0, test_y0 = train_test_split(X0,y0,test_size=0.2, shuffle=False)

train_X0.insert(0, 'ones', 1)
# train_X1 = train_X1.to_numpy()
test_X0.insert(0, 'ones', 1)


start = time.time()
w0 = w_normaleqn(train_X0,train_y0)

print('Time =',time.time() - start)

train_mseloss0 = mseloss_normaleqn(train_X0,w0,train_y0)[0][0]
print('Train mse loss \t=', train_mseloss0)

test_mseloss0 = mseloss_normaleqn(test_X0,w0,test_y0)[0][0]
print('Test mse loss \t=', test_mseloss0)


y_pred = f_pred(train_X0, w0)
r2_train0 = r2_score(train_y0, y_pred)
print('Train R2 score \t=', r2_train0)

y_pred = f_pred(test_X0, w0)
r2_test0 = r2_score(test_y0, y_pred)
print('Test R2 score \t=', r2_test0)

LINEAR REGRESSION - train0:


LinAlgError: Singular matrix

### Train 1

In [7]:
print('LINEAR REGRESSION - train1:')
X1 = train1.copy()
X1.drop(['Id', 'SalePrice'], axis=1, inplace=True)
y1 = train1['SalePrice'][:,None]

train_X1, test_X1, train_y1, test_y1 = train_test_split(X1,y1,test_size=0.2, shuffle=False)

train_X1.insert(0, 'ones', 1)
# train_X1 = train_X1.to_numpy()
test_X1.insert(0, 'ones', 1)


start = time.time()
w1 = w_normaleqn(train_X1,train_y1)

print('Time =',time.time() - start)

train_mseloss1 = mseloss_normaleqn(train_X1,w1,train_y1)[0][0]
print('Train mse loss \t=', train_mseloss1)

test_mseloss1 = mseloss_normaleqn(test_X1,w1,test_y1)[0][0]
print('Test mse loss \t=', test_mseloss1)


y_pred = f_pred(train_X1, w1)
r2_train1 = r2_score(train_y1, y_pred)
print('Train R2 score \t=', r2_train1)

y_pred = f_pred(test_X1, w1)
r2_test1 = r2_score(test_y1, y_pred)
print('Test R2 score \t=', r2_test1)

LINEAR REGRESSION - train1:
Time = 0.006913185119628906
Train mse loss 	= 724503657.0670046
Test mse loss 	= 2118527244.1644464
Train R2 score 	= 0.8827229771923898
Test R2 score 	= 0.6685651754863232


### Train 2

In [24]:
print('LINEAR REGRESSION - train2:')
X2 = train2.copy()
X2.drop(['Id', 'SalePrice'], axis=1, inplace=True)
y2 = train2['SalePrice'][:,None]

train_X2, test_X2, train_y2, test_y2 = train_test_split(X2,y2,test_size=0.2, shuffle=False)

train_X2.insert(0, 'ones', 1)
test_X2.insert(0, 'ones', 1)

start = time.time()
w2 = w_normaleqn(train_X2,train_y2)

print('Time =',time.time() - start)

train_mseloss2 = mseloss_normaleqn(train_X2,w2,train_y2)[0][0]
print('Train mse loss \t=', train_mseloss2)

test_mseloss2 = mseloss_normaleqn(test_X2,w2,test_y2)[0][0]
print('Test mse loss \t=', test_mseloss2)

y_pred = f_pred(train_X2, w2)
r2_train2 = r2_score(train_y2, y_pred)
print('Train R2 score \t=', r2_train2)

y_pred = f_pred(test_X2, w2)
r2_test2 = r2_score(test_y2, y_pred)
print('Test R2 score \t=', r2_test2)

LINEAR REGRESSION - train2:
Time = 0.00598454475402832
Train mse loss 	= 695841434655724.8
Test mse loss 	= 1032364255420290.4
Train R2 score 	= -142081.82923167179
Test R2 score 	= -277463.67234785407


### Train 3

In [25]:
print('LINEAR REGRESSION - train3:')
X3 = train3.copy()
X3.drop(['Id', 'SalePrice'], axis=1, inplace=True)
y3 = train3['SalePrice'][:,None]

train_X3, test_X3, train_y3, test_y3 = train_test_split(X3,y3,test_size=0.2, shuffle=False)

train_X3.insert(0, 'ones', 1)
test_X3.insert(0, 'ones', 1)

start = time.time()
w3 = w_normaleqn(train_X3,train_y3)

print('Time =',time.time() - start)

train_mseloss3 = mseloss_normaleqn(train_X3,w3,train_y3)[0][0]
print('Train mse loss \t=', train_mseloss3)

test_mseloss3 = mseloss_normaleqn(test_X3,w3,test_y3)[0][0]
print('Test mse loss \t=', test_mseloss3)


y_pred = f_pred(train_X3, w3)
r2_train3 = r2_score(train_y3, y_pred)
print('Train R2 score \t=', r2_train3)

y_pred = f_pred(test_X3, w3)
r2_test3 = r2_score(test_y3, y_pred)
print('Test R2 score \t=', r2_test3)

LINEAR REGRESSION - train3:
Time = 0.00595545768737793
Train mse loss 	= 2069026058727.2686
Test mse loss 	= 2178612184427.163
Train R2 score 	= -421.47135846900505
Test R2 score 	= -584.5374328889659


### Train 4

In [26]:
print('LINEAR REGRESSION - train4:')
X4 = train4.copy()
X4.drop(['Id', 'SalePrice'], axis=1, inplace=True)
y4 = train4['SalePrice'][:,None]

train_X4, test_X4, train_y4, test_y4 = train_test_split(X4,y4,test_size=0.2, shuffle=False)

train_X4.insert(0, 'ones', 1)
test_X4.insert(0, 'ones', 1)

start = time.time()
w4 = w_normaleqn(train_X4,train_y4)

print('Time =',time.time() - start)

train_mseloss4 = mseloss_normaleqn(train_X4,w4,train_y4)[0][0]
print('Train mse loss \t=', train_mseloss4)

test_mseloss4 = mseloss_normaleqn(test_X4,w4,test_y4)[0][0]
print('Test mse loss \t=', test_mseloss4)


y_pred = f_pred(train_X4, w4)
r2_train4 = r2_score(train_y4, y_pred)
print('Train R2 score \t=', r2_train4)

y_pred = f_pred(test_X4, w4)
r2_test4 = r2_score(test_y4, y_pred)
print('Test R2 score \t=', r2_test4)

LINEAR REGRESSION - train4:
Time = 0.005948543548583984
Train mse loss 	= 169765854502.49716
Test mse loss 	= 167239064517.9596
Train R2 score 	= -33.664237731949925
Test R2 score 	= -43.94821667507848


In [ ]:
# X0_full = X0.copy()
# X0_full.insert(0, 'ones', 1)
# w0 = w_normaleqn(X0_full,y0)
# mseloss0 = mseloss_normaleqn(X0_full,w0,y0)#[0][0]
# print('mse loss0:', mseloss0)

# X1_full = X1.copy()
# X1_full.insert(0, 'ones', 1)
# w1 = w_normaleqn(X1_full,y1)
# mseloss1 = mseloss_normaleqn(X1_full,w1,y1)#[0][0]
# print('mse loss1:', mseloss1)

# X2_full = X2.copy()
# X2_full.insert(0, 'ones', 1)
# w2 = w_normaleqn(X2_full,y2)
# mseloss2 = mseloss_normaleqn(X2_full,w2,y2)#[0][0]
# print('mse loss2:', mseloss2)

# X3_full = X3.copy()
# X3_full.insert(0, 'ones', 1)
# w3 = w_normaleqn(X3_full,y3)
# mseloss3 = mseloss_normaleqn(X3_full,w3,y3)#[0][0]
# print('mse loss3:', mseloss3)

# X4_full = X4.copy()
# X4_full.insert(0, 'ones', 1)
# w4 = w_normaleqn(X4_full,y4)
# mseloss4 = mseloss_normaleqn(X4_full,w4,y4)#[0][0]
# print('mse loss4:', mseloss4)